In [ ]:
%matplotlib inline

from IPython.display import display, HTML
from ipywidgets import interactive
import matplotlib.pyplot as plt
# import networkx as nx
import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sns
import sklearn.metrics
# import tensorflow as tf
# import tensorflow_probability as tfp
from tqdm.notebook import tqdm

from collections import namedtuple, defaultdict, Counter
import functools
from glob import glob
import itertools
import os
import pickle
import re
import socket
import subprocess
import sys
from time import sleep

plt.style.use('seaborn')
# tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
np.set_printoptions(suppress=True, precision=2, threshold=20)
sys.path += ['../src/']

display(HTML("<h3>Runned on %s</h3><h4>Last commit: %s</h4><h4>Conda env: %s</h4>" % (
    socket.gethostname(),
    subprocess.run(["git","log", "-1", "--pretty=%h %s"],
        check=False, capture_output=True).stdout.decode() or 'none.',
    (re.findall(""".*envs[\\\/](.*?)[\\\/]""", sys.executable) or [sys.executable])[0]
)))


In [2]:
df_all = pd.read_csv("../data/labm-experiments/main-experiments-learnable-abm.csv")
print(set(df_all["tags.mlflow.source.git.commit"].values))
df_all = df_all[df_all.status == "FINISHED"]
if 'tags.crashed' in df_all.columns:
    df_all = df_all[df_all['tags.crashed'].isna()]
params = [c[7:] for c in df_all.columns if c.startswith("params.") ]
df_all.columns = [c[7:] if c.startswith("params.") else c for c in df_all.columns]
print(len(df_all))

{'2a4ecb25997a2822cc9b10841572ca436a908aa3'}
9


In [3]:
deltas = (1/16., )
stddevs = [1.]
num_Dbs = [256]

In [4]:
df_all = df_all[df_all.learning_rate == 0.001].copy()
df_all = df_all[df_all.delta.isin(deltas)].copy()
df_all = df_all[df_all.num_deals_error_stddev.isin(stddevs)]
df_all = df_all[df_all.num_considered_Dbs.isin(num_Dbs)]
len(df_all)

9

In [5]:
validation_traces = []
# validation_traces = sorted(list(df_all.seed.unique()))[:10]
# validation_traces

In [6]:
for num_deals_error_stddev in stddevs:
    for num_considered_Dbs in num_Dbs:
            for seed in validation_traces:
                exp_string = f"{num_considered_Dbs}, {num_deals_error_stddev}, seed {seed}"
                df_experiment = df_all[
                            (df_all['num_deals_error_stddev'] == num_deals_error_stddev) &
                            (df_all['num_considered_Dbs'] == num_considered_Dbs) &
                            (df_all['seed'] == seed)
                      ]
                num_experiments = len(df_experiment)
                if num_experiments == 0:
                    print("MISSING:", exp_string)
                elif num_experiments > 1:
                    print("Duplicate:", exp_string)
                    print(set(df_experiment["tags.mlflow.source.git.commit"].values))

In [7]:
params_not_unique = dict()
params_unique = dict()
for f in params:
    values = set(df_all[f])
    if len(values) > 1:
        params_not_unique[f] = values
    else:
        params_unique[f] = values

display(HTML("<h1>Parameters Not Unique</h1>"))
display(pd.DataFrame(index=params_not_unique.keys(), data=params_not_unique.values()).fillna(''))

display(HTML("<h1>Unique Parameters</h1>"))
display(pd.DataFrame(index=params_unique.keys(), data=params_unique.values(), columns=["Value"]).fillna(''))

,0,1,2,3,4,5,6,7,8
seed,53,54,55,56,57,58,59,60,61


,Value
delta,0.0625
num_considered_Dbs,256
Gammak,[0.5 0.4 0.1]
learning_steps,4
N,1000
M0_initialization_variance,1
epochs,5
use_relative_error,False
threshold,0.05
price_error_stddev,1.0


In [8]:
print(df_all.num_restarts.unique())
assert df_all.num_restarts.min() > 1
df_best_runs = df_all

[10]


In [9]:
df_best_runs['metrics.pearson_MP'] = (
    df_best_runs['metrics.pearson_M'] + df_best_runs['metrics.pearson_P']) / 2

df_best_runs['metrics.r2_score_MP'] = (
    df_best_runs['metrics.r2_score_M'] + df_best_runs['metrics.r2_score_P']) / 2

In [10]:
# metric = 'metrics.pearson_M'

# results = np.full((len(num_Dbs), len(stddevs)), np.nan)

# for j, num_deals_error_stddev in enumerate(stddevs):
#     for i, num_Db in enumerate(num_Dbs):
#         results[i, j] = df_best_runs[
#             (df_best_runs['num_deals_error_stddev'] == num_deals_error_stddev) &
#             (df_best_runs['num_considered_Dbs'] == num_Db) &
#             (df_best_runs['seed'].isin(validation_traces))
#         ][metric].mean()

# fig, ax = plt.subplots()
# vmin = 0.
# vmax = np.ceil(np.nanmax(results) * 10) / 10
# ax.matshow(results, cmap='Blues', vmin=vmin, vmax=vmax)
# for (i, j), z in np.ndenumerate(results):
#     z = np.floor(z * 100) / 100
#     ax.text(j, i, '{:0.2f}'.format(z), ha='center', va='center', color='w', fontsize=20)
# plt.grid()
# plt.xticks(np.arange(len(stddevs)), stddevs)
# plt.xlabel('$\epsilon_D$')
# plt.yticks(np.arange(len(num_Dbs)), num_Dbs)
# plt.ylabel('Num. of $D^B$ samples')
# # plt.gca().xaxis.set_label_position('top')
# ax.xaxis.set_ticks_position('bottom')
# plt.title(r'$\rho(M, \hat{M})$' if metric == 'metrics.pearson_M' else metric)

# for ext in ("pdf", "png"):
#     plt.savefig(f"../paper/plots/hyperparam-learnable-{metric.split('.')[1]}.{ext}",
#                         bbox_inches='tight', dpi=200)
# plt.show()

In [11]:
best_num_deals_error_stddev = 1.0
best_num_considered_Dbs = 256

In [12]:
# test_traces = sorted(list(df_all.seed.unique()))[10:]
test_traces = sorted(list(df_all.seed.unique()))
assert not (set(test_traces) & set(validation_traces))
df_plot = df_best_runs[
    (df_best_runs['num_deals_error_stddev'] == best_num_deals_error_stddev) &
    (df_best_runs['num_considered_Dbs'] == best_num_considered_Dbs) &
    (df_best_runs['seed'].isin(test_traces))
].copy()

for f in sorted(df_all.columns):
    if 'metrics.' in f:
        print(f"{f:40}: {df_plot[f].mean():.4f}")

metrics.explained_variance_score_Db     : 0.7727
metrics.explained_variance_score_M      : 0.5718
metrics.explained_variance_score_M0     : 0.5222
metrics.explained_variance_score_Nd     : 0.9988
metrics.explained_variance_score_P      : 0.9485
metrics.final_est_num_agents            : 4998.4444
metrics.final_num_agents                : 4992.7778
metrics.max_error_Db                    : 17.0527
metrics.max_error_M                     : 504.1943
metrics.max_error_M0                    : 504.1943
metrics.max_error_Nd                    : 1.0277
metrics.max_error_P                     : 1.3752
metrics.mean_absolute_error_Db          : 4.9531
metrics.mean_absolute_error_M           : 99.7290
metrics.mean_absolute_error_M0          : 145.1152
metrics.mean_absolute_error_Nd          : 0.0730
metrics.mean_absolute_error_P           : 0.5945
metrics.mean_squared_error_Db           : 43.6120
metrics.mean_squared_error_M            : 20204.2336
metrics.mean_squared_error_M0           : 43808.36

In [13]:
df_plot['dist_to_median'] = (
    df_plot['metrics.pearson_M'] - df_plot['metrics.pearson_M'].median()).abs()

In [14]:
df_plot[['run_id', 'seed', 'metrics.pearson_M', 'dist_to_median']
       ].sort_values('dist_to_median')

,run_id,seed,metrics.pearson_M,dist_to_median
4,7d6eaf3270c84eb28e728dfeb88dd7e4,58,0.777123,0.000000
7,9aed690323a74e859f2cb75d194de277,55,0.763976,0.013147
1,798fa399a46446bd91e18f2b7d14d48e,61,0.753489,0.023634
8,2b4bd9f02ec04fdabc9071392b3e072b,54,0.752143,0.024980
2,b69623f5d69f468bb831509c0ce1042a,60,0.837652,0.060529
6,9ce2780f25dd46e2a00f2698b2b3f9d9,56,0.888171,0.111047
5,c21d1153f1084d14a8dc4819b36c9f33,57,0.888416,0.111293
9,8d62c5b5219c4b59a116751c5937df16,53,0.952923,0.175800
3,a959206bfffc43d3a169f2fabd920343,59,0.479772,0.297351


In [15]:
df_plot["Model"] = "learnable"

In [16]:
existing_df_plot = pd.read_csv("../data/main-figure-experiments.csv")

In [17]:
existing_df_plot = existing_df_plot[existing_df_plot.Model != 'learnable']

In [18]:
pd.concat([existing_df_plot, df_plot],
    join="inner",
    ignore_index=True,
).to_csv("../data/main-figure-experiments.csv", index=False)